In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [9]:
PREPROCESSED_DIR = r"C:\Users\gamer\OneDrive\Desktop\IIT Project\Datasets\datasets\Pilot\preprocessed_image"
CSV_PATH = r"C:\Users\gamer\OneDrive\Desktop\IIT Project\Datasets\datasets\Pilot\train_final.csv"
IMG_SIZE = 512

In [5]:
def apply_noise_reduction(image):
    denoised = cv2.medianBlur(image, 3)
    denoised = cv2.GaussianBlur(denoised, (3, 3), 0)
    return denoised

def apply_normalization(image):
    img = image.astype('float32') / 255.0
    mean, std = np.mean(img), np.std(img)
    return (img - mean) / (std + 1e-7)

# 2. Build Extractor (Your Code)
def build_feature_extractor():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model.output
    feature_vector = GlobalAveragePooling2D()(x)
    return Model(inputs=base_model.input, outputs=feature_vector)

In [11]:
from openvino.tools.ovc.moc_frontend import extractor
feature_extractor = build_feature_extractor()
# 1. Load and CLEAN the Dataframe
train_df = pd.read_csv(CSV_PATH)
# CRITICAL FIX: Remove any leading/trailing spaces from the filenames
train_df['Image Index'] = train_df['Image Index'].str.strip()

# 2. Path Debugging (Checks if the folder is actually visible to Python)
if not os.path.exists(PREPROCESSED_DIR):
    print(f"❌ ERROR: The directory '{PREPROCESSED_DIR}' does not exist.")
else:
    sample_files = os.listdir(PREPROCESSED_DIR)[:5]
    print(f"✅ Folder found! Sample files in folder: {sample_files}")

# 3. Extraction Loop
train_features = []
processed_indices = []

print(f"🚀 Starting Extraction for {len(train_df)} images...")

for index, row in train_df.iterrows():
    img_name = row['Image Index']
    # Use abspath to ensure Python isn't getting confused by relative paths
    img_path = os.path.abspath(os.path.join(PREPROCESSED_DIR, img_name))

    if os.path.exists(img_path):
        # A. Load Grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"❌ Could not read image: {img_name} (File might be corrupted)")
            continue

        # B. Apply Preprocessing
        img = apply_noise_reduction(img)
        img = apply_normalization(img)

        # C. Prep for DenseNet (Resize & convert to 3-channel RGB)
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_rgb = cv2.merge([img_resized, img_resized, img_resized])
        img_input = np.expand_dims(img_rgb, axis=0)

        # D. Extract Features
        feat = feature_extractor.predict(img_input, verbose=0)
        train_features.append(feat.flatten())
        processed_indices.append(index)

        if len(processed_indices) % 100 == 0:
            print(f"✅ Extracted features for {len(processed_indices)} images...")
    else:
        # Debugging print to see exactly what Python is looking for
        if index < 5: # Only print first 5 failures to avoid clutter
            print(f"⚠️ Not Found: Checked path -> {img_path}")

# Final Wrap up
if train_features:
    train_features_arr = np.array(train_features)
    # Save the features so you don't have to extract them again!
    np.save('extracted_features_test.npy', train_features_arr)
    print(f"✨ Success! Saved feature matrix of shape {train_features_arr.shape}")
else:
    print("❌ No images were processed. Please check if the filenames in your CSV include '.png'.")

✅ Folder found! Sample files in folder: ['00000003_000.png', '00000003_001.png', '00000003_002.png', '00000003_003.png', '00000003_004.png']
🚀 Starting Extraction for 28080 images...
✅ Extracted features for 100 images...
✅ Extracted features for 200 images...
✅ Extracted features for 300 images...
✅ Extracted features for 400 images...
✅ Extracted features for 500 images...
✅ Extracted features for 600 images...
✅ Extracted features for 700 images...
✅ Extracted features for 800 images...
✅ Extracted features for 900 images...
✅ Extracted features for 1000 images...
✅ Extracted features for 1100 images...
✅ Extracted features for 1200 images...
✅ Extracted features for 1300 images...
✅ Extracted features for 1400 images...
✅ Extracted features for 1500 images...
✅ Extracted features for 1600 images...
✅ Extracted features for 1700 images...
✅ Extracted features for 1800 images...
✅ Extracted features for 1900 images...
✅ Extracted features for 2000 images...
✅ Extracted features for 2